In [4]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2


In [5]:
!python -V

Python 3.10.12


In [6]:
import os
import pickle
import pandas as pd
import numpy as np

In [7]:
!pwd

/home/gurlich/mlops-zoomcamp/cohorts/2024/04-deployment/solution


In [28]:
# datasets located in "data/" folder from working directory
# homework artifacts located in "cohorts/2024/04-deployment/solution/"
HOMEWORK_FOLDER = "/home/gurlich/mlops-zoomcamp/cohorts/2024/04-deployment/solution"
HOMEWORK_FOLDER = "."

In [29]:
with open(f'{HOMEWORK_FOLDER}/model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/gurlich/mlops-zoomcamp/mlops_venv/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/gurlich/mlops-zoomcamp/mlops_venv/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [16]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [18]:
# df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_????-??.parquet')
df = read_data('~/mlops-zoomcamp/data/yellow_tripdata_2023-03.parquet')

In [19]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [20]:
y_pred

array([16.24590642, 26.1347962 , 11.88426424, ..., 11.59533603,
       13.11317847, 12.89999218])

### Q1 Calculate standart deviation

In [31]:
stnd_dev = np.std(y_pred)
print(f"Standart deviation of the predicted duration: {stnd_dev:.2f}")

Standart deviation of the predicted duration: 6.25


### Q2 Preparing the output

In [22]:
year = 2023
month = 3

In [23]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [24]:
df_result = pd.DataFrame()

df_result['ride_id'] = df['ride_id']
df_result['predict'] = y_pred

In [25]:
output_dir = f"{HOMEWORK_FOLDER}/output"
# create directory if not exists
os.makedirs(output_dir, exist_ok=True)

output_file = f"predict-{year:04d}-{month:02d}.parquet"
file_path = output_dir+'/'+output_file

In [26]:
df_result.to_parquet(
    file_path,
    engine='pyarrow',
    compression=None,
    index=False
)

In [33]:

file_size = os.path.getsize(file_path)
print(f"File size '{output_file}': {file_size/1024/1024:.2f} MB")

File size 'predict-2023-03.parquet': 65.16 MB


### Q5 mean predicted duration

In [36]:
mean_duration = np.mean(y_pred)
print(f"Mean predicted duration: {mean_duration:.2f}")

Mean predicted duration: 14.20
